TF is loaded.

In [1]:
import sys, os, csv, collections

In [2]:
from tf.fabric import Fabric

DATABASE = '~/github'
XBIB = 'extrabiblical'

TF = Fabric(locations=[DATABASE], modules=[XBIB], silent=False )

This is Text-Fabric 3.0.3
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

72 features found and 0 ignored


In [4]:
api = TF.load('''
    otype mother lex typ code function rela det txt prs kind vs vt sp book chapter verse label language
''')

  0.00s loading features ...
   |     0.00s Feature overview: 65 for nodes; 4 for edges; 3 configs; 7 computed
  0.05s All features loaded/computed - for details use loadLog()


In [5]:
api.loadLog()
api.makeAvailableIn(globals())

   |     0.00s M otext                from C:/Users/Martijn/github/extrabiblical
   |     0.00s B otype                from C:/Users/Martijn/github/extrabiblical
   |     0.02s B oslots               from C:/Users/Martijn/github/extrabiblical
   |     0.00s M otext                from C:/Users/Martijn/github/extrabiblical
   |     0.00s B book                 from C:/Users/Martijn/github/extrabiblical
   |     0.00s B chapter              from C:/Users/Martijn/github/extrabiblical
   |     0.00s B verse                from C:/Users/Martijn/github/extrabiblical
   |     0.01s B g_cons               from C:/Users/Martijn/github/extrabiblical
   |     0.01s B g_lex_utf8           from C:/Users/Martijn/github/extrabiblical
   |     0.00s B g_suffix             from C:/Users/Martijn/github/extrabiblical
   |     0.01s B g_suffix_utf8        from C:/Users/Martijn/github/extrabiblical
   |     0.00s B g_word               from C:/Users/Martijn/github/extrabiblical
   |     0.01s B g_word_utf8

In a few helper functions is is calculated whether a clause is a main clause, a subordinate clause (3 different varieties) or undecided.

In [6]:
# %load main_sub
def in_dep_calc(cl):  
      
    in_dep = ''        
    if F.rela.v(cl) == 'ReSu': # is the clause resumptive?
        moth_obj = E.mother.f(cl)[0]
        in_dep = rela_calc(moth_obj)
    else:
        in_dep = rela_calc(cl) # does the clause have a dependent CCR?

    if in_dep == '':
    	words = L.d(cl, 'word') # is there a wayyiqtol?
    	for word in words:
            if F.vt.v(word) == 'wayq':
                in_dep += 'Main'
                        
    if in_dep == '':  # if everything else does not give a result, we look at the CARC
        cl_atoms = L.d(cl, 'clause_atom')
        in_dep = carc_calc(cl_atoms)
        
    return(in_dep)


#####################################


def carc_calc(cl_atoms):
    in_dep_c = ''
    carc = F.code.v(cl_atoms[0])
    if 999 > int(carc) > 499:
        in_dep_c += 'SubAdv'
    elif int(carc) in {0, 999}:
        in_dep_c = 'Main'
    elif 17 > int(carc) > 9:
        in_dep_c += 'SubAdv'
    elif 75 > int(carc) > 50:
        in_dep_c += 'SubAdv'
    elif 168 > int(carc) > 99:
        in_dep_c += 'Main'
    elif 500 > int(carc) > 299:
        in_dep_c += 'Main'
    else:
        in_dep_c += 'Undc'
        
    return(in_dep_c)

###################################################################################################

def rela_calc(cl):
    in_dep_r = ''
    ccr = F.rela.v(cl)
    if ccr in {'Subj', 'Objc', 'Cmpl', 'PreC', 'Voct', 'Frnt'}:
        in_dep_r += 'SubArg'
    elif ccr in {'Attr', 'RgRc', 'Spec'}:
        in_dep_r += 'SubMod'
    elif ccr in {'Adju', 'PrAd'}:
        in_dep_r += 'SubAdv'
    elif ccr == 'Coor':
        moth_obj = E.mother.f(cl)[0]
        if F.otype.v(moth_obj) in {'word', 'phrase'}:
            in_dep_r += 'SubMod'
        else:
            while F.rela.v(moth_obj) == 'Coor':
                moth_obj = E.mother.f(moth_obj)[0]
            ccr = F.rela.v(cl)
            if ccr in {'Subj', 'Objc', 'Cmpl', 'PreC', 'Voct', 'Frnt'}:
                in_dep_r += 'SubArg'
            elif ccr in {'Attr', 'RgRc', 'Spec'}:
                in_dep_r += 'SubMod'
            elif ccr in {'Adju', 'PrAd'}:
                in_dep_r += 'SubAdv'
                
        if in_dep_r == '':
            if F.otype.v(moth_obj) != 'clause':
                in_dep_r += 'SubMod'
            else:
                cl_atoms = L.d(moth_obj, 'clause_atom')
                in_dep_r = carc_calc(cl_atoms)
                
    return(in_dep_r)

############################################################################################


The function mother_clause_calc determines the tense of the mother of that clause(atom).

In [7]:
def mother_tense_calc(cl):
    
    mother_tense = ''
    cl_atoms = L.d(cl, 'clause_atom')
    moth = E.mother.f(cl_atoms[0])
    if len(moth) == 0:
        mother_tense += 'no_mother'
    else: 
        if F.otype.v(moth[0]) in {'word', 'phrase'}:
            mother_tense += F.otype.v(moth[0])
        else:
            cl = L.u(moth[0], 'clause')[0]
            if F.kind.v(cl) == 'NC':
                mother_tense += 'nominal'
            elif F.kind.v(cl) == 'WP':
                mother_tense += 'no_pred'
                        
            else:
                phrases = L.d(cl, 'phrase')
                pred = False
                prec = False
                for phr in phrases:
                    if F.function.v(phr) in {'Pred', 'PreS', 'PreO'}:
                        pred = True
                        pred_phr = phr
                    elif F.function.v(phr) in {'PreC', 'PtcO'}:
                        prec = True
                        prec_phr = phr
                if pred == True:
                    words = L.d(pred_phr, 'word')
                    for word in words:
                        if F.sp.v(word) == 'verb':
                            mother_tense += F.vt.v(word)
                elif prec == True:
                    words = L.d(prec_phr, 'word')
                    for word in words:
                        if F.sp.v(word) == 'verb':
                            mother_tense += F.vt.v(word)
                            
    return(mother_tense)

A list with all the different phrase functions is created (in this case copied from the biblical data). This is used in the main program.

In [8]:
extra_cl_funcs = ['Adju', 'Cmpl', 'Conj', 'EPPr', 'ExsS', 'Exst', 'Frnt', 'IntS', 'Intj', 'Loca', 'ModS', 'Modi', 'NCoS', 'NCop', 'Nega', 'Objc', 'PrAd', 'PrcS', 'PreO', 'PreS', 'PtcO', 'Ques', 'Rela', 'Supp', 'Time', 'Voct', 'Unkn']

This is the main program. Of each clause it it determined whether it is a copula clause and its information is stored in the dict cop_clauses.

In [9]:
cop_cl_list = []
cop_clauses = {}

for cl in F.otype.s('clause'):
    feat_list = []
    cl_kind = F.kind.v(cl)
    if cl_kind in {'NC', 'VC'}: 
        phrases = L.d(cl, 'phrase')
        phr_funcs = [F.function.v(phr) for phr in phrases]
        
        if 'Subj' in phr_funcs and 'PreC' in phr_funcs:
            cl_type = ''
            if cl_kind == 'VC' and 'Pred' in phr_funcs:
                cl_type += 'hyh'
            elif cl_kind == 'NC':
                cl_type += 'nom'
                
            if cl_type in {'hyh', 'nom'}:
                
                # collect basic info about clause
                feat_list.append(cl_type) # type of clause: hyh or nom
                feat_list.append(str(cl)) # clause id
                bo, ch, ve = T.sectionFromNode(cl)
                feat_list.append(bo) # book
                feat_list.append(str(ch)) # chapter
                feat_list.append(str(ve)) # verse
                feat_list.append(F.typ.v(cl)) # clause type of database
                feat_list.append(F.rela.v(cl) + '_') # clause relation
                
                # eg if book is '1_Samuel', add 'Samuel' to feat_list, relevant for biblical data only
                feat_list.append(bo)
                
                # ebh_lbh
                if bo in {'1QM', '1QS'}:
                    feat_list.append('Qumranic')
                else:
                    feat_list.append('Epigraphic')
                feat_list.append('prose') #genre
                feat_list.append(F.txt.v(cl)[-1]) # Q, D, N
            
                if phr_funcs.index('Subj') < phr_funcs.index('PreC'): #subj and prec order
                    feat_list.append('SP')
                else:
                    feat_list.append('PS')
                    
                # which conjunction
                phr_types = [F.typ.v(phr) for phr in phrases]
                if 'CP' in phr_types: 
                    conj = phrases[phr_types.index('CP')]
                    words = L.d(conj, 'word')
                    words_lex = '_'.join([F.lex.v(wo) for wo in words])
                    feat_list.append(words_lex)
                else:
                    feat_list.append('no_conj')
                
                # clause length, in verbal clauses the Pred is not counted
                if cl_kind == 'VC':
                    feat_list.append(str(len(phrases) - 1)) 
                else:
                    feat_list.append(str(len(phrases)))
                
                subj_pos = phr_funcs.index('Subj')
                feat_list.append(F.typ.v(phrases[subj_pos])) #phrase type of subject
                feat_list.append(F.det.v(phrases[subj_pos])) #determination of subject
                
                # length of subj in words
                words_subj = L.d(phrases[subj_pos], 'word') # length of subject in words
                subj_len = len(words_subj)
                
                # the following is uncommented in the biblical data file, the suffix is not counted as words automatically there
                #for wo in words_subj:                    
                #    if not F.prs.v(wo) in {'absent', 'n/a'}:
                #        subj_len += 1
                feat_list.append(str(subj_len))
                
                prec_pos = phr_funcs.index('PreC')
                feat_list.append(F.typ.v(phrases[prec_pos])) #phrase type of prec
                feat_list.append(F.det.v(phrases[prec_pos])) #determination of prec
                
                # length of prec in words
                words_prec = L.d(phrases[prec_pos], 'word')
                prec_len = len(words_prec)
                
                # just as with the subject, uncomment the following for biblical data
                #for wo in words_prec:                    
                #    if not F.prs.v(wo) in {'absent', 'n/a'}:
                #        prec_len += 1
                feat_list.append(str(prec_len))
            
                # main or subordinate clause
                feat_list.append(in_dep_calc(cl))
                                     
                # negation in clause
                phr_types = [F.typ.v(phr) for phr in phrases]
                if 'NegP' in phr_types:
                    feat_list.append('neg')
                else:
                    feat_list.append('non_neg')
                
                # aramaic or hebrew
                words = L.d(cl, 'word')
                feat_list.append(F.language.v(words[0]))
                
                # subgenre (list or instruction)
                feat_list.append('_')
                
                # collect info about other phrases in clause (present (1) or absent (0))
                for item in extra_cl_funcs:   
                    if item in phr_funcs:
                        feat_list.append('1')
                    else:
                        feat_list.append('0')
                
                # tense ot the mother of the clause_atom
                feat_list.append(mother_tense_calc(cl))
                
                cop_cl_list.append(cl)
                cop_clauses[cl] = feat_list                

Information is saved in a csv file.

In [11]:
csvh = open(r"hyh_nom_extrabiblical.csv", "w")
header = ['cl_type', 'cl_id', 'book', 'chapter', 'verse', 'clause_type_etcbc', 'clause_rela', 'book2', 'ebh_lbh', 'genre', 'txt_type', 's_p_order', 'CP', 'cl_len', 'subj_type', 'subj_det', 'subj_len', 'pc_type', 'pc_det', 'pc_len', 'main_sub', 'nega', 'language', 'subgenre']
for item in extra_cl_funcs:
    header.append(item)
header.append('mother')
csvh.write('{}\n'.format(','.join(header)))

for key in cop_cl_list:
    features = cop_clauses[key]
    csvh.write('{}\n'.format(','.join(features)))
    
csvh.close()